In [ ]:
#Height of ridge adjusted

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import tkinter as tk
from tkinter import filedialog, simpledialog, Scale, messagebox, OptionMenu, StringVar, BooleanVar
from scipy.stats import norm
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib

matplotlib.use('TkAgg')

# Lattice Boltzmann parameters
nx, ny = 300, 100  # Grid size (width, height)
tau = 1.0  # Relaxation time
omega = 1 / tau  # Collision term
dx = dy = 1
dt = 1
rho0 = 1.0  # Initial density
max_time_steps = 1000

# Velocity set for D2Q9 model
c_sqr = 1/3  # Speed of sound squared
w = np.array([4/9] + [1/9]*4 + [1/36]*4)  # Weights (length 9)
c = np.array([
    [0, 0], [1, 0], [0, 1], [-1, 0], [0, -1],
    [1, 1], [-1, 1], [-1, -1], [1, -1]
])  # Directions (9 rows)

# Global variables for simulation state
f = np.ones((9, nx, ny)) * rho0 / 9
rho = np.sum(f, axis=0)
u = np.zeros((2, nx, ny))
ridges_rotated = np.zeros((nx, ny), dtype=bool)
current_time_step = 0
current_ridge_type = "Parallel Lines" # Default ridge type
ridge_height_factor = 0.1 # Fixed ridge height factor for visualization
roughness_factor = 0.1 # Fixed roughness factor

def gaussian_profile(y, center, sigma):
    """Generates a 1D Gaussian profile."""
    return norm.pdf(y, loc=center, scale=sigma)

def apply_roughness(ridges, roughness_factor):
    """Applies slight roughness to the edges of the ridges."""
    rough_ridges = np.copy(ridges)
    for x in range(nx):
        for y in range(ny):
            # Check neighbors for potential roughness
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx_neighbor, ny_neighbor = x + dx, y + dy
                    if 0 <= nx_neighbor < nx and 0 <= ny_neighbor < ny:
                        # Add a pixel if a neighbor is a ridge and current is not
                        if ridges[nx_neighbor, ny_neighbor] == 1 and ridges[x, y] == 0:
                            if np.random.rand() < roughness_factor:
                                rough_ridges[x, y] = 1
                        # Remove a pixel if current is a ridge and neighbor is not (to create jaggedness)
                        elif ridges[x, y] == 1 and ridges[nx_neighbor, ny_neighbor] == 0:
                            if np.random.rand() < roughness_factor:
                                if np.sum(ridges[max(0, x - 1):min(nx, x + 2), max(0, y - 1):min(ny, y + 2)]) > 1: # Ensure not isolating a ridge
                                    rough_ridges[x, y] = 0
    return rough_ridges

def generate_parallel_ridges(roughness=roughness_factor):
    """Generates parallel horizontal ridge lines with roughness."""
    ridges = np.zeros((nx, ny))
    for i in range(ny):
        if (i // 10) % 2 == 0:
            ridges[:, i] = 1
    return apply_roughness(ridges, roughness)

def generate_wavy_ridges(amplitude=3, frequency=0.1, roughness=roughness_factor):
    """Generates wavy horizontal ridge lines with roughness."""
    ridges = np.zeros((nx, ny))
    for i in range(ny):
        if (i // 10) % 2 == 0:
            for x in range(nx):
                offset = int(amplitude * np.sin(x * frequency))
                if 0 <= i + offset < ny:
                    ridges[x, i + offset] = 1
    return apply_roughness(ridges, roughness)

def generate_whorl_ridges(center_x=None, center_y=None, spacing=10, roughness=roughness_factor):
    """Generates a basic whorl-like ridge pattern using concentric circles with roughness."""
    ridges = np.zeros((nx, ny))
    if center_x is None:
        center_x = nx // 2
    if center_y is None:
        center_y = ny // 2

    for x in range(nx):
        for y in range(ny):
            radius = np.sqrt((x - center_x)**2 + (y - center_y)**2)
            if int(radius) % spacing == 0:
                ridges[x, y] = 1
    return apply_roughness(ridges, roughness)

def generate_arch_ridges(frequency=0.05, amplitude=20, roughness=roughness_factor):
    """Generates a basic arch-like ridge pattern with roughness."""
    ridges = np.zeros((nx, ny))
    spacing = 10
    for i in range(spacing, ny - spacing, spacing):
        center_x = nx // 2
        curve = amplitude * np.sin(np.linspace(-np.pi / 2, np.pi / 2, nx))
        offset = i - ny // 2
        y_coords = (ny // 2 + curve + offset).astype(int)
        for x in range(nx):
            if 0 <= y_coords[x] < ny:
                ridges[x, y_coords[x]] = 1
    return apply_roughness(ridges, roughness)

def generate_loop_ridges(center_x=None, center_y=None, radius=30, spacing=8, roughness=roughness_factor):
    """Generates a basic loop-like ridge pattern with roughness."""
    ridges = np.zeros((nx, ny))
    if center_x is None:
        center_x = int(nx * 0.3)
    if center_y is None:
        center_y = ny // 2

    for angle in np.linspace(0, 2 * np.pi, 100):
        x_loop = int(radius * np.cos(angle) + center_x)
        y_loop = int(radius * np.sin(angle) + center_y)
        for i in range(-5, 6): # Thicken the loop line
            x_thick = x_loop + i
            if 0 <= x_thick < nx and 0 <= y_loop < ny:
                ridges[x_thick, y_loop] = 1

    # Add parallel lines entering the loop
    for y in range(0, ny, spacing):
        for x in range(0, int(nx * 0.4)):
            if not ridges[x, y]: # Don't overwrite the loop
                ridges[x, y] = 1
    return apply_roughness(ridges, roughness)


def initialize_simulation(rotation_angle_degrees, spray_angle_degrees, tip_to_surface_distance, spray_width_factor=1.0, spray_profile_type='gaussian', ridge_type="Parallel Lines"):
    """Initializes the simulation fields and ridge pattern."""
    global f, rho, u, ridges_rotated, current_ridge_type
    current_ridge_type = ridge_type

    # Generate ridge pattern based on the selected type
    if ridge_type == "Parallel Lines":
        ridges = generate_parallel_ridges()
    elif ridge_type == "Wavy Lines":
        ridges = generate_wavy_ridges()
    elif ridge_type == "Whorl":
        ridges = generate_whorl_ridges()
    elif ridge_type == "Arch":
        ridges = generate_arch_ridges()
    elif ridge_type == "Loop":
        ridges = generate_loop_ridges()
    # Add more ridge types here if needed

    # Rotate the ridge pattern
    img_ridges = Image.fromarray(ridges.astype(np.uint8))
    rotated_img = img_ridges.rotate(rotation_angle_degrees, expand=False, fillcolor=0) # Keep size
    ridges_rotated[:] = np.array(rotated_img) > 0.5

    # Initialize fields
    f[:] = np.ones((9, nx, ny)) * rho0 / 9
    rho[:] = np.sum(f, axis=0)
    u[:] = 0

    # Set DESI spray parameters
    angle_radians = spray_angle_degrees * np.pi / 180
    ux_spray = 0.02 * np.cos(angle_radians)
    uy_spray_base = 0.02 * np.sin(angle_radians)

    # Apply spray velocity at left boundary
    center_y = ny / 2
    sigma_base = ny / 60 * (tip_to_surface_distance - 1) + ny / 20  # Calculate base sigma
    sigma = sigma_base * spray_width_factor # Adjust sigma based on width factor
    y_coords = np.arange(ny)

    for y in range(ny):
        u[0, 0, y] = ux_spray # Apply uniform x-velocity
        if spray_profile_type == 'gaussian':
            profile = gaussian_profile(y_coords, center_y, sigma)
            profile /= np.max(profile) # Normalize
            u[1, 0, y] = uy_spray_base * profile[y] # Modulate y-velocity with Gaussian profile
        elif spray_profile_type == 'uniform':
            if abs(y - center_y) < sigma * 1.5: # A simple way to define a uniform-like profile
                u[1, 0, y] = uy_spray_base
            else:
                u[1, 0, y] = 0
        # Add other profile types here if needed

def equilibrium(rho, u):
    """Calculate equilibrium distribution function."""
    cu = np.einsum('ia,axy->ixy', c, u)
    usqr = u[0]**2 + u[1]**2
    feq = np.einsum('i,jk->ijk', w, rho) * (1 + 3*cu + 9/2*cu**2 - 3/2*usqr)
    return feq

def update_flow(time_step):
    """Updates the flow for the given number of time steps (optimized outflow)."""
    global f, rho, u, current_time_step
    current_time_step = 0 # Reset time step when initializing from GUI

    target_step = int(time_step)

    for _ in range(target_step - current_time_step):
        if current_time_step >= max_time_steps:
            break
        # Compute equilibrium
        feq = equilibrium(rho, u)

        # Collision step
        f += omega * (feq - f)

        # Streaming step
        for i, ci in enumerate(c):
            f[i] = np.roll(f[i], ci, axis=(0, 1))

        # Apply outflow boundary condition at the right edge (x = nx - 1)
        f[3, nx - 1, :] = f[3, nx - 2, :]  # c[3] = [-1, 0] (west)
        f[6, nx - 1, :] = f[6, nx - 2, :]  # c[6] = [-1, 1] (northwest)
        f[7, nx - 1, :] = f[7, nx - 2, :]  # c[7] = [-1, -1] (southwest)

        # Recompute macroscopic variables
        rho = np.maximum(np.sum(f, axis=0), 1e-6)  # Prevent zero density
        u[0] = np.sum(f * c[:, 0, None, None], axis=0) / rho
        u[1] = np.sum(f * c[:, 1, None, None], axis=0) / rho

        # Enforce bounce-back on ridges (zero velocity) - This is a simplified way
        u[:, ridges_rotated] = 0  # Corrected bounce-back condition

        current_time_step += 1
    plot_flow()

def plot_flow():
    """Plots the current state of the solvent flow."""
    global fig, canvas, u, ridges_rotated, current_ridge_type, ridge_height_factor

    plt.clf()
    plt.imshow(u[0].T, cmap='jet', origin='lower', extent=[0, nx, 0, ny])
    plt.colorbar(label='Solvent Flow Velocity')
    plt.xlabel("X-axis (Flow Direction)")
    plt.ylabel("Y-axis (Fingerprint Surface)")
    plt.title(f"DESI Solvent Flow Over {current_ridge_type} (t={current_time_step})")

    if show_ridges_var.get():
        ridge_y, ridge_x = np.where(ridges_rotated.T) # Transpose for correct indexing
        # Plot ridges with a fixed linewidth of 0.1
        unique_ridge_y = np.unique(ridge_y)
        for y_val in unique_ridge_y:
            x_indices = ridge_x[ridge_y == y_val]
            if len(x_indices) > 0:
                plt.plot(x_indices, np.ones_like(x_indices) * y_val, color='white', linewidth=ridge_height_factor, alpha=0.7) # Removed the label part
        if unique_ridge_y.size > 0 and show_ridges_var.get(): # Keep the legend only if ridges are shown
            plt.legend()

    canvas.draw()

def start_simulation_from_gui(selected_ridge_type):
    rotation = rotation_angle_scale.get()
    spray_angle = spray_angle_scale.get()
    tip_distance = tip_to_surface_distance_scale.get()
    spray_width_factor = spray_width_factor_scale.get()
    spray_profile_type = spray_profile_type_var.get()
    global current_time_step
    current_time_step = 0
    initialize_simulation(rotation, spray_angle, tip_distance, spray_width_factor, spray_profile_type, selected_ridge_type)
    time_step_slider.config(to=max_time_steps)
    update_flow(0) # Show initial state

def on_ridge_type_change(selected_ridge_type):
    """Callback function when the ridge type is changed in the dropdown."""
    start_simulation_from_gui(selected_ridge_type)

def update_schematic(spray_angle, tip_distance, spray_width_factor, spray_profile_type='gaussian'):
    schematic_canvas.delete("all")  # Clear previous drawing
    surface_y = schematic_canvas_height * 0.8
    surface_height = 20
    surface_x_start = 20
    surface_x_end = schematic_canvas_width - 20
    schematic_canvas.create_rectangle(surface_x_start, surface_y, surface_x_end, surface_y + surface_height, fill='lightgray', outline='black')
    schematic_canvas.create_text((surface_x_start + surface_x_end) / 2, surface_y + surface_height + 10, text='Surface', anchor=tk.CENTER)

    spray_angle_rad = np.radians(float(spray_angle)) # Ensure angle is float
    tip_distance_float = float(tip_distance) # Ensure distance is float
    spray_width_factor_float = float(spray_width_factor) # Ensure width factor is float

    sprayer_length = 80
    tip_y = surface_y - tip_distance_float * 10  # Scale tip distance for visualization
    tip_x = 50

    end_x = tip_x + sprayer_length * np.cos(spray_angle_rad)
    end_y = tip_y - sprayer_length * np.sin(spray_angle_rad)

    schematic_canvas.create_line(tip_x, tip_y, end_x, end_y, width=2)
    schematic_canvas.create_oval(tip_x - 5, tip_y - 5, tip_x + 5, tip_y + 5, fill='blue') # Sprayer tip

    # Label angles and distances (simplified)
    schematic_canvas.create_text(tip_x + 30, tip_y - 20, text=f"α = {spray_angle}°", anchor=tk.W)
    schematic_canvas.create_line(tip_x, tip_y, tip_x, surface_y, dash=(4, 4))
    # Convert tip_distance to float here
    tip_distance_float_for_label = float(tip_distance)
    schematic_canvas.create_text(tip_x + 10, (tip_y + surface_y) / 2, text=f"d1 = {tip_distance_float_for_label:.1f} mm", anchor=tk.W)

    # Represent spray width (very simplified)
    spray_width_vis = 20 * spray_width_factor_float
    schematic_canvas.create_line(tip_x, tip_y, tip_x + spray_width_vis * np.sin(spray_angle_rad), tip_y + spray_width_vis * np.cos(spray_angle_rad), width=1, fill='gray')
    schematic_canvas.create_line(tip_x, tip_y, tip_x - spray_width_vis * np.sin(spray_angle_rad), tip_y + spray_width_vis * np.cos(spray_angle_rad), width=1, fill='gray')

    # Indicate roughness (very symbolic)
    if roughness_factor > 0:
        for i in range(5):
            offset_x = np.random.randint(-5, 6)
            offset_y = np.random.randint(-2, 3)
            schematic_canvas.create_line(surface_x_start + 50 + i * 10 + offset_x, surface_y + offset_y, surface_x_start + 50 + (i + 1) * 10 + offset_x, surface_y + np.random.randint(-2, 3), width=1)

def download_plot():
    """Saves the current plot as a PNG file."""
    file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                               filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
    if file_path:
        fig.savefig(file_path)
        messagebox.showinfo("Download Successful", f"Plot saved to {file_path}")

# --- GUI Setup ---
root = tk.Tk()
root.title("DESI Solvent Flow Simulator")

# --- Schematic Window ---
schematic_window = tk.Toplevel(root)
schematic_window.title("DESI Emitter Schematic")
schematic_canvas_width = 300
schematic_canvas_height = 200
schematic_canvas = tk.Canvas(schematic_window, width=schematic_canvas_width, height=schematic_canvas_height, bg='white')
schematic_canvas.pack()

# Ridge Type Control
ridge_type_label = tk.Label(root, text="Fingerprint Ridge Type:")
ridge_type_label.pack(pady=2) # Reduced padding

ridge_type_var = StringVar(root)
ridge_type_var.set("Parallel Lines") # Default value
ridge_type_options = ["Parallel Lines", "Wavy Lines", "Whorl", "Arch", "Loop"] # Add "Arch" and "Loop"
ridge_type_menu = OptionMenu(root, ridge_type_var, *ridge_type_options, command=on_ridge_type_change)
ridge_type_menu.pack(pady=2) # Reduced padding

# Rotation Angle Control
rotation_angle_label = tk.Label(root, text="Ridge Pattern Rotation Angle (°):")
rotation_angle_label.pack(pady=2) # Reduced padding
rotation_angle_scale = Scale(root, from_=-180, to=180, orient=tk.HORIZONTAL, length=300, resolution=1, label="Rotation Angle")
rotation_angle_scale.set(0) # Default rotation angle
rotation_angle_scale.pack(pady=2) # Reduced padding
rotation_angle_scale.config(command=lambda angle: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get(), spray_width_factor_scale.get(), spray_profile_type_var.get()))

# Spray Angle Control
spray_angle_label = tk.Label(root, text="DESI Sprayer Angle (°):")
spray_angle_label.pack(pady=2) # Reduced padding
spray_angle_scale = Scale(root, from_=1, to=179, orient=tk.HORIZONTAL, length=300, resolution=1, label="Spray Angle")
spray_angle_scale.set(72) # Default spray angle
spray_angle_scale.pack(pady=2) # Reduced padding
spray_angle_scale.config(command=lambda angle: update_schematic(angle, tip_to_surface_distance_scale.get(), spray_width_factor_scale.get(), spray_profile_type_var.get()))

# Tip to Surface Distance (d1) Control
tip_to_surface_distance_label = tk.Label(root, text="Tip to Surface Distance (d1, mm):")
tip_to_surface_distance_label.pack(pady=2) # Reduced padding
tip_to_surface_distance_scale = Scale(root, from_=1, to=10, orient=tk.HORIZONTAL, length=300, resolution=0.1, label="Tip Distance (d1)")
tip_to_surface_distance_scale.set(5) # Default tip distance
tip_to_surface_distance_scale.pack(pady=2) # Reduced padding
# Modify the command here to ensure tip_distance is a float
tip_to_surface_distance_scale.config(command=lambda distance: update_schematic(spray_angle_scale.get(), distance, spray_width_factor_scale.get(), spray_profile_type_var.get()))

# Spray Width Factor Control
spray_width_factor_label = tk.Label(root, text="Spray Width Factor:")
spray_width_factor_label.pack(pady=2) # Reduced padding
spray_width_factor_scale = Scale(root, from_=0.5, to=2.0, orient=tk.HORIZONTAL, length=300, resolution=0.1, label="Spray Width")
spray_width_factor_scale.set(1.0) # Default spray width factor
spray_width_factor_scale.pack(pady=2) # Reduced padding
spray_width_factor_scale.config(command=lambda width: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get(), width, spray_profile_type_var.get()))

# Spray Profile Type Control
spray_profile_type_label = tk.Label(root, text="Spray Profile Type:")
spray_profile_type_label.pack(pady=2) # Reduced padding
spray_profile_type_var = StringVar(root)
spray_profile_type_var.set("gaussian") # Default value
spray_profile_type_options = ["gaussian", "uniform"] # Add more if implemented
spray_profile_type_menu = OptionMenu(root, spray_profile_type_var, *spray_profile_type_options)
spray_profile_type_menu.pack(pady=2) # Reduced padding
spray_profile_type_var.trace_add("write", lambda *args: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get(), spray_width_factor_scale.get(), spray_profile_type_var.get()))

# Toggle for Showing Ridges
show_ridges_var = BooleanVar()
show_ridges_checkbox = tk.Checkbutton(root, text="Show Ridges", variable=show_ridges_var, command=plot_flow)
show_ridges_checkbox.pack(pady=2) # Reduced padding
show_ridges_var.set(True) # Default to showing ridges

# Time Step Control
time_step_label = tk.Label(root, text="Flow Time Step:")
time_step_label.pack(pady=2) # Reduced padding
time_step_slider = Scale(root, from_=0, to=max_time_steps, orient=tk.HORIZONTAL, length=300, resolution=1, label="Time Step", command=update_flow)
time_step_slider.set(0)
time_step_slider.pack(pady=2) # Reduced padding

# Start Simulation Button
start_button = tk.Button(root, text="Initialize Simulation", command=lambda: start_simulation_from_gui(ridge_type_var.get()))
start_button.pack(pady=5) # Reduced padding

# Download PNG Button
download_button = tk.Button(root, text="Download PNG", command=download_plot)
download_button.pack(pady=5) # Reduced padding

# --- Matplotlib Plot in Tkinter ---
fig, ax = plt.subplots(figsize=(8, 4))
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack(pady=5) # Reduced padding

# Initial drawing of the schematic
update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get(), spray_width_factor_scale.get(), spray_profile_type_var.get())

# Initialize simulation with the default ridge type
start_simulation_from_gui(ridge_type_var.get())

root.mainloop()